Downloading the data in python and repeating all of the steps again

In [2]:
import json,urllib2
import pandas as pd
import numpy as np
import folium
import re
import json
# Do not reexecute
# add a header so that the website doesnt give 403 error
opener = urllib2.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
response = opener.open('https://api.phila.gov/bike-share-stations/v1')

# parse json using pandas
df = pd.read_json(response)

In [3]:
# the data has 105 features, one for each station
df['features'][0]

{u'geometry': {u'coordinates': [-75.16374, 39.95378], u'type': u'Point'},
 u'properties': {u'addressCity': u'Philadelphia',
  u'addressState': u'PA',
  u'addressStreet': u'1401 John F. Kennedy Blvd.',
  u'addressZipCode': u'19102',
  u'bikesAvailable': 19,
  u'closeTime': u'23:58:00',
  u'docksAvailable': 3,
  u'eventEnd': None,
  u'eventStart': None,
  u'isEventBased': False,
  u'isVirtual': False,
  u'kioskId': 3004,
  u'kioskPublicStatus': u'Active',
  u'name': u'Municipal Services Building Plaza',
  u'openTime': u'00:02:00',
  u'publicText': u'',
  u'timeZone': u'Eastern Standard Time',
  u'totalDocks': 25,
  u'trikesAvailable': 0},
 u'type': u'Feature'}

In [4]:
len(df['features'])

105

In [5]:
# Total number of bikes or docks available

total_bikes = 0.0
total_docks = 0.0
aval_docks = 0.0

for i in range(len(df['features'])):
    total_bikes +=df['features'][i]['properties']['bikesAvailable']
    total_docks +=df['features'][i]['properties']['totalDocks']
    aval_docks  +=df['features'][i]['properties']['docksAvailable']
    
print("We currently have %d number of bikes" %total_bikes)
print("We currently have %d number of docks" %aval_docks)
print("We have %d docks totally in the system" %total_docks)

ratio = total_bikes/total_docks

print("The ratio of bikes to total docks in the system is %f" %ratio)

We currently have 758 number of bikes
We currently have 1231 number of docks
We have 2013 docks totally in the system
The ratio of bikes to total docks in the system is 0.376552


In [6]:
# Calculating the amount of demand or supply for each station, same as in previous solution

DorS = []
for i in range(len(df['features'])):
    DorS.append(int
    (round(df['features'][i]['properties']['totalDocks']*ratio)-df['features'][i]['properties']['bikesAvailable']))

print(DorS[0:10])

[-10, -3, -2, 6, -8, -3, 1, 1, 1, 1]


If the _ith_ element of the list DorS has a positive value, it is a __demand__ station

If the _ith_ element of the list DorS has a negative value, it is a __supply__ station

In [7]:
# creating different lists of demand and supply

demand = [0]*len(DorS)
supply = [0]*len(DorS)

i = 0
while i < len(DorS):
    if DorS[i] > 0:
        demand[i] = DorS[i]
        supply[i] = 0
    elif DorS[i] < 0:
        demand[i] = 0
        supply[i] = -1*DorS[i]
    else:
        demand[i] = 0
        supply[i] = 0
    i+=1
        
print(demand[1:20])
print(supply[1:20])

[0, 0, 6, 0, 0, 1, 1, 1, 1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[3, 2, 0, 8, 3, 0, 0, 0, 0, 0, 2, 2, 1, 5, 2, 14, 12, 11, 8]


In [8]:
# Visualizing the results

philly = [39.9626, -75.1652]

maps = folium.Map(location = philly, tiles = 'OpenStreetMap', zoom_start = 13)

coordinates = []

for i in range(0,len(df['features'])):
    coordinates.append(((df["features"][i]["geometry"]['coordinates'])))

coordinates = [tuple([i[1],i[0]]) for i in coordinates]

for i in range(len(df['features'])):
    if supply[i] > 0:
        folium.Marker(coordinates[i],popup="Supply Station",icon = folium.Icon(color = "red")).add_to(maps)

for i in range(len(df['features'])):
    if demand[i] > 0:
        folium.Marker(coordinates[i],popup="Demand Station",icon = folium.Icon(color = "blue")).add_to(maps)        
maps

In [41]:
# making the data file for AMPL

# set demand
semicol = ";"
setdem = "param dem := "
#setdem = setdem + "\n"+ str(demand) + semicol
#setdem = setdem.replace(",","")
#setdem = re.sub(r"[\[]", "", setdem)
#setdem = re.sub(r"[\]]", "", setdem)
print(setdem)
i = 0
while i < len(demand):
    print i, "\t", demand[i]
    i+=1

print(semicol)
print("\n")

# set supply
setsup = "param sup := "
#setsup = setsup + str(supply) + semicol
#setsup = setsup.replace(",","")
##setsup = re.sub(r"[\[]", "", setsup)
#setsup = re.sub(r"[\]]", "", setsup)
#print(setsup)
print(setsup)
i = 0
while i < len(supply):
    print i, "\t", supply[i]
    i+=1

print(semicol)


# values of number of stations(n), number of supply or demand stations(n) and truck capacity
# value of m

dem = 0
for i in demand:
    if i > 0:
        dem+=1

print("\n")
m = "param m := " + str(dem) + semicol
print(m)

# value of n
n = "param n := " + str(len(df['features'])) + semicol
print("\n")
print(n)

# value of z, here we have arbitarily assigned the value of Z to be 50 i.e. the truck can carry 50 bikes
z = "param z := 25" + semicol
print("\n")
print(z)
r = "param r := " + stratio + semicol
print("\n")
print(r)

param dem := 
0 	0
1 	1
2 	0
3 	2
4 	0
5 	0
6 	0
7 	2
8 	7
9 	0
10 	8
11 	0
12 	0
13 	0
14 	0
15 	0
16 	0
17 	0
18 	0
19 	0
20 	4
21 	0
22 	0
23 	3
24 	0
25 	0
26 	8
27 	0
28 	4
29 	5
30 	0
31 	4
32 	0
33 	0
34 	2
35 	0
36 	0
37 	3
38 	5
39 	4
40 	0
41 	0
42 	0
43 	0
44 	3
45 	0
46 	0
47 	0
48 	5
49 	0
50 	1
51 	0
52 	0
53 	0
54 	2
55 	2
56 	7
57 	3
58 	3
59 	1
60 	0
61 	6
62 	4
63 	6
64 	2
65 	0
66 	0
67 	0
68 	4
69 	5
70 	0
71 	3
72 	6
73 	1
74 	5
75 	0
76 	0
77 	0
78 	4
79 	0
80 	5
81 	8
82 	3
83 	1
84 	0
85 	3
86 	4
87 	0
88 	0
89 	1
90 	2
91 	0
92 	5
93 	0
94 	4
95 	0
96 	1
97 	0
98 	5
99 	0
100 	0
101 	0
102 	0
103 	6
104 	0
;


param sup := 
0 	2
1 	0
2 	0
3 	0
4 	8
5 	5
6 	1
7 	0
8 	0
9 	2
10 	0
11 	0
12 	2
13 	0
14 	5
15 	0
16 	15
17 	14
18 	10
19 	4
20 	0
21 	1
22 	0
23 	0
24 	2
25 	8
26 	0
27 	7
28 	0
29 	0
30 	0
31 	0
32 	1
33 	4
34 	0
35 	4
36 	0
37 	0
38 	0
39 	0
40 	1
41 	3
42 	5
43 	0
44 	0
45 	5
46 	1
47 	4
48 	0
49 	9
50 	0
51 	3
52 	3
53 	5
54 	0
55 	0
56 	0
57 	0
58

TypeError: cannot concatenate 'str' and 'instance' objects

In [37]:
%%capture cap --no-stderr
print(setdem)
i = 0
while i < len(demand):
    print i+1, "\t", demand[i] + df['features'][i]['properties']['bikesAvailable']
    i+=1
print(semicol)
print("\n")
print(setsup)
i = 0
while i < len(supply):
    print i+1, "\t", df['features'][i]['properties']['totalDocks']
    i+=1

print(semicol)
print("\n")
print(m)
print("\n")
print(n)
print("\n")
print(z)
print("\n")
print(r)
with open('output.dat', 'w') as f:
    f.write(cap.stdout)

#### The model for making groups is

### Minimize Z = $\sum\limits_{j = 1}^m Y_j$

Subject to
1. $\sum\limits_{j=1}^n X_{ij} = 1 \quad\forall$  i $\in$ {1..n}

2. $\sum\limits_{i=1}^n X_{ij} <= n*Y_i \quad\forall$  j $\in$ {1..m}

3. $\sum\limits_{i=1}^n X_{ij} *$ Supply at node i $<= Z \quad\forall$  j $\in {1..m}$

4. $\sum\limits_{i=1}^n X_{ij} *$ Demand at node i $<= \sum\limits_{i=1}^n X_{ij} *$ Supply at node i $\quad\forall$ j $\in$ {1..m}

In [19]:
%load_ext ampl

The ampl extension is already loaded. To reload it, use:
  %reload_ext ampl


In [21]:
%%ampl
reset;

option solver "/home/dinesh/Downloads/amplide/gurobi";

param n;
param m;
param z;

param dem{1..n};
param sup{1..n};

data output.dat;

var x{i in 1..n, j in 1..m} binary;
var y{j in 1..m} binary;

minimize objfun:
    sum{j in 1..m}y[j];

subject to C1{i in 1..n}:
    sum{j in 1..m}x[i,j] = 1;
    
subject to C2{j in 1..m}:
    sum{i in 1..n}x[i,j] <= n*y[j];

subject to C3{j in 1..m}:
    sum{i in 1..n}x[i,j] * dem[i] <= z;

subject to C4{j in 1..m}:
    sum{i in 1..n}x[i,j] * dem[i] >= sum{i in 1..n}x[i,j] * sup[i] * 0.4;
    
solve;

display x;
display y;

print {i in 1..n}: {j in 1..m} x[i,j] > ./indego_x11.csv;
print {j in 1..m} y[j] > ./indego_y11.csv;

Gurobi 7.0.0: optimal solution; objective 5
2835 simplex iterations
7 branch-and-cut nodes
x [*,*]
:     1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19 :=
1     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
2     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
3     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
4     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
5     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
6     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
7     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
8     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
9     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
10    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
11    0   0   0   

#### The results  produce a data object called *instance* which I do not have an idea of how to index. I posted the code on stack overflow, but the data is not editable

#### I got the results to be available in a file as shown above, and got it back into the system to make it readable as a data frame again